In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Đọc dữ liệu từ file CSV
train_file = "D:/Du An/Data FLow/v1/data/train.csv"
test_file = "D:/Du An/Data FLow/v1/data/test.csv"

df_train = pd.read_csv(train_file)
df_test = pd.read_csv(test_file)

# Chuyển đổi cột Date thành kiểu datetime
df_train['Date'] = pd.to_datetime(df_train['Date'])
df_test['Date'] = pd.to_datetime(df_test['Date'])

# Thêm thông tin ngày trong tuần
df_train['DayOfWeek'] = df_train['Date'].dt.dayofweek
df_test['DayOfWeek'] = df_test['Date'].dt.dayofweek

# Sắp xếp theo thời gian
df_train.sort_values(by='Date', inplace=True)
df_test.sort_values(by='Date', inplace=True)

# Chọn dữ liệu doanh thu và số lượng bán theo ngày
sales_train = df_train[['Date', 'Revenue', 'Units', 'DayOfWeek']].groupby('Date').sum()
sales_test = df_test[['Date', 'Revenue', 'Units', 'DayOfWeek']].groupby('Date').sum()

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler(feature_range=(0, 1))
sales_train_scaled = scaler.fit_transform(sales_train)
sales_test_scaled = scaler.transform(sales_test)

# Tạo tập dữ liệu cho LSTM
def create_lstm_dataset(data, time_step=30):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), :])
        y.append(data[i + time_step, :])
    return np.array(X), np.array(y)

time_step = 30
X_train, y_train = create_lstm_dataset(sales_train_scaled, time_step)
X_test, y_test = create_lstm_dataset(sales_test_scaled, time_step)

# Xây dựng mô hình Bi-Directional LSTM
model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True, input_shape=(time_step, X_train.shape[2]))),
    Dropout(0.2),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(32, return_sequences=False)),
    Dropout(0.2),
    Dense(y_train.shape[1])
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Thêm EarlyStopping và ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)

# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping, reduce_lr])

# Dự báo
y_pred = model.predict(X_test)

# Chuyển đổi dữ liệu về dạng gốc
y_test_inv = scaler.inverse_transform(np.pad(y_test, ((0, 0), (0, 1)), mode='constant'))[:, :-1]
y_pred_inv = scaler.inverse_transform(np.pad(y_pred, ((0, 0), (0, 1)), mode='constant'))[:, :-1]

# Vẽ biểu đồ
dates = sales_test.index[time_step:]
plt.figure(figsize=(12, 6))
plt.plot(dates, y_test_inv[:, 0], label='Thực tế (Revenue)')
plt.plot(dates, y_pred_inv[:, 0], label='Dự báo (Revenue)')
plt.xlabel('Date')
plt.ylabel('Revenue')
plt.legend()
plt.show()

c:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
116/116 ━━━━━━━━━━━━━━━━━━━━ 50s 172ms/step - loss: 0.0049 - val_loss: 0.0015 - learning_rate: 0.0010
Epoch 2/100
116/116 ━━━━━━━━━━━━━━━━━━━━ 16s 140ms/step - loss: 0.0042 - val_loss: 0.0015 - learning_rate: 0.0010
Epoch 3/100
116/116 ━━━━━━━━━━━━━━━━━━━━ 17s 148ms/step - loss: 0.0042 - val_loss: 0.0015 - learning_rate: 0.0010
Epoch 4/100
116/116 ━━━━━━━━━━━━━━━━━━━━ 17s 144ms/step - loss: 0.0048 - val_loss: 0.0014 - learning_rate: 0.0010
Epoch 5/100
116/116 ━━━━━━━━━━━━━━━━━━━━ 16s 136ms/step - loss: 0.0041 - val_loss: 0.0014 - learning_rate: 0.0010
Epoch 6/100
116/116 ━━━━━━━━━━━━━━━━━━━━ 17s 146ms/step - loss: 0.0038 - val_loss: 0.0014 - learning_rate: 0.0010
Epoch 7/100
116/116 ━━━━━━━━━━━━━━━━━━━━ 18s 154ms/step - loss: 0.0038 - val_loss: 0.0014 - learning_rate: 0.0010
Epoch 8/100
116/116 ━━━━━━━━━━━━━━━━━━━━ 18s 151ms/step - loss: 0.0040 - val_loss: 0.0013 - learning_rate: 0.0010
Epoch 9/100
116/116 ━━━━━━━━━━━━━━━━━━━━ 17s 144ms/step - loss: 0.0033 - val_loss: 0.001

ValueError: operands could not be broadcast together with shapes (505,4) (3,) (505,4) 

In [ ]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Tạo các đặc trưng thời gian
def create_features(df):
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    return df

df_train = create_features(df_train)
df_test = create_features(df_test)

# Chọn đặc trưng và target
features = ['Year', 'Month', 'Day', 'DayOfWeek', 'Units']
target = 'Revenue'

# Chuẩn hóa dữ liệu đầu vào
scaler = MinMaxScaler()
df_train[features] = scaler.fit_transform(df_train[features])
df_test[features] = scaler.transform(df_test[features])

# Chia tập dữ liệu
X_train, X_val, y_train, y_val = train_test_split(df_train[features], df_train[target], test_size=0.2, random_state=42)

# Khởi tạo mô hình Random Forest
rf_model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)

# Huấn luyện
rf_model.fit(X_train, y_train)

# Dự báo
y_pred_rf = rf_model.predict(X_val)

# Đánh giá mô hình
mae = mean_absolute_error(y_val, y_pred_rf)
mse = mean_squared_error(y_val, y_pred_rf)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred_rf)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")


In [ ]:
# Sắp xếp dữ liệu theo thời gian
df_train.sort_values(by='Date', inplace=True)
df_test.sort_values(by='Date', inplace=True)

# Tạo đặc trưng thời gian
def create_features(df):
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    return df

df_train = create_features(df_train)
df_test = create_features(df_test)

# Tạo lag features (lấy giá trị doanh thu của ngày trước đó làm input)
def create_lag_features(df, lag_days=[1, 7, 14]):
    for lag in lag_days:
        df[f'Revenue_Lag{lag}'] = df['Revenue'].shift(lag)
    return df

df_train = create_lag_features(df_train)
df_test = create_lag_features(df_test)

# Loại bỏ các hàng có giá trị NaN do shift()
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

# Chọn các cột đầu vào và đầu ra
features = ['Year', 'Month', 'Day', 'DayOfWeek', 'Units', 'Revenue_Lag1', 'Revenue_Lag7', 'Revenue_Lag14']
target = 'Revenue'

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
df_train[features] = scaler.fit_transform(df_train[features])
df_test[features] = scaler.transform(df_test[features])

# Chia tập dữ liệu train và validation
X_train, X_val, y_train, y_val = train_test_split(df_train[features], df_train[target], test_size=0.2, random_state=42, shuffle=False)

# Khởi tạo mô hình XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', 
                             n_estimators=200, 
                             learning_rate=0.05, 
                             max_depth=6, 
                             subsample=0.8, 
                             colsample_bytree=0.8, 
                             random_state=42)

# Huấn luyện mô hình
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=True)

# Dự báo
y_pred_xgb = xgb_model.predict(X_val)

# Đánh giá mô hình
mae = mean_absolute_error(y_val, y_pred_xgb)
mse = mean_squared_error(y_val, y_pred_xgb)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred_xgb)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")

In [ ]:
# Tạo đặc trưng thời gian
def create_features(df):
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    return df

df_train = create_features(df_train)
df_test = create_features(df_test)

# Tạo lag features (giá trị doanh thu của ngày trước đó làm input)
def create_lag_features(df, lag_days=[1, 7, 14]):
    for lag in lag_days:
        df[f'Revenue_Lag{lag}'] = df['Revenue'].shift(lag)
    return df

df_train = create_lag_features(df_train)
df_test = create_lag_features(df_test)

# Loại bỏ các hàng NaN do shift()
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

# Chọn các cột đặc trưng và nhãn
features = ['Year', 'Month', 'Day', 'DayOfWeek', 'Units', 'Revenue_Lag1', 'Revenue_Lag7', 'Revenue_Lag14']
target = 'Revenue'

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
df_train[features] = scaler.fit_transform(df_train[features])
df_test[features] = scaler.transform(df_test[features])

# Chia dữ liệu thành tập train và validation
X_train, X_val, y_train, y_val = train_test_split(df_train[features], df_train[target], test_size=0.2, random_state=42, shuffle=False)

# Khởi tạo mô hình LightGBM
lgb_model = lgb.LGBMRegressor(
    boosting_type='gbdt', 
    objective='regression', 
    n_estimators=500, 
    learning_rate=0.05, 
    max_depth=6, 
    num_leaves=31,
    subsample=0.8, 
    colsample_bytree=0.8,
    random_state=42
)

# Huấn luyện mô hình
lgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='rmse', early_stopping_rounds=20, verbose=10)

# Dự báo
y_pred_lgb = lgb_model.predict(X_val)

# Đánh giá mô hình
mae = mean_absolute_error(y_val, y_pred_lgb)
mse = mean_squared_error(y_val, y_pred_lgb)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred_lgb)

print(f"📌 MAE: {mae}")
print(f"📌 RMSE: {rmse}")
print(f"📌 R-squared: {r2}")

# Trực quan hóa kết quả
import matplotlib.pyplot as plt

dates = df_train['Date'].iloc[-len(y_val):]  # Lấy ngày tương ứng với tập validation
plt.figure(figsize=(12, 6))
plt.plot(dates, y_val.values, label='Thực tế (Revenue)', color='blue')
plt.plot(dates, y_pred_lgb, label='Dự báo (Revenue)', color='red')
plt.xlabel('Date')
plt.ylabel('Revenue')
plt.legend()
plt.show()